# Preprocessing


In [3]:
# Unzip files in their respective folders
from scripts.unzip_files import unzip_files_KGS 

source_folder = '../data/zip_files'
destination_folder = '../data/v1.0_raw_data'

unzip_files_KGS(source_folder, destination_folder)

In [2]:
# Download las files from KGS
from scripts.download_las_files import download_and_organize_las_files

source_folder = '../data/v1.0_raw_data'  
destination_folder = '../data/v2.0_zip_files'

# File to map las file names to urls
database_path = '../data/zip_files/ks_las_files.txt'  

# Download and organize las files
download_and_organize_las_files(source_folder, destination_folder, database_path)

Jonescanyonsoutheast |                                                  
Schaben              |                                                  
Marcotte             |                                                  
Cutter               |                                                  
Medicinelodgenorth   |                                                  
Kraftprusa           |                                                  
Wellington           |                                                  
Burrton              |                                                  
Garnettshoestring    |                                                  
Bemisshutts          |                                                  
Bradshawgasarea      |                                                  
Webster              |                                                  
Morel                |                                                  
Eldorado             |                             

In [4]:
from scripts.ajust_las_files import process_all_las_files

source_folder = '../data/v2.0_zip_files'
destination_folder = '../data/v3.0_las_files'
csv_folder = '../data/v1.0_raw_data'

process_all_las_files(source_folder, destination_folder, csv_folder)



In [5]:
# Exploratory Data Aanlysis
import os
from scripts.unzip_files import unzip_files
from scripts.exploratory_data_analysis import create_project_from_directory, create_curve_well_matrix, remap

source_folder = '../data/v2.0_zip_files'
destination_folder = '../data/v3.0_data_filtered'
field_folder = '../data/v3.0_data_filtered/JonesCanyonSoutheast'

# Specify the folder containing .las files for a single field
field_folder = '../data/v3.0_data_filtered/JonesCanyonSoutheast'




## atento modificar la funcion para que lea el zip file y no las file. 
## a trabajar de una con el ajuste del punto las

# PENDIENTES:
### Description de curvas 
### litologia de la formacion
### estas dos mejoras a realizar usando LLM y RAG ya que es procesamiento de texto


In [7]:
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming 'create_curve_well_matrix' is your function that generates the DataFrame
curve_well_matrix = create_curve_well_matrix(project)

# Now use this DataFrame with sns.heatmap
plt.figure(figsize=(20, 10))  # Adjust the size as needed
sns.heatmap(curve_well_matrix, cmap="YlGnBu", cbar=False)
plt.show()


NameError: name 'project' is not defined